# Importing necessities
 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import cv2
import os
import posenet
import time
import pandas as pd
import numpy as np
import pickle


# Loading Random Forest model

In [2]:
filename_RFC = "RFC.pickle"
filename_ANN = "ANN"
filename_CNN = "CNN"

### Reading One Hot Encoding categories

In [3]:
with open("OneHotEncoding categories.txt", 'r') as f:
    ohe_categories = f.readline()
ohe_categories = list(ohe_categories.split("'")[1:-1])
while " " in ohe_categories:
    ohe_categories.remove(" ")
print(ohe_categories)

['actions1', 'fall', 'pushups', 'sit', 'walking', 'walking-with-hands-up']


 **Configuring my cudnn for tensorflow 1** <br><br>
 **Needed for my laptop, unsure for others**

In [4]:
global sess
global graph

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

graph = tf.get_default_graph()

set_session(sess)

model_RFC = pickle.load(open(filename_RFC, 'rb'))
model_ANN = tf.keras.models.load_model(filename_ANN)
model_CNN = tf.keras.models.load_model(filename_CNN)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
#session = tf.keras.backend.get_session()
#init = tf.global_variables_initializer()
#session.run(init)

# Prediction

In [6]:
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
temporary = pd.DataFrame()

#with tf.Session() as sess:
with graph.as_default():
    
    set_session(sess)
        
    # Setting ouput stride to 16
    output_stride = 16
    model_id = 101
    model_cfg, model_outputs = posenet.load_model(model_id, sess)

    cap = cv2.VideoCapture("data\\online_fall.mp4")
    # cap = cv2.VideoCapture(0)
    input_image, display_image, output_scale = posenet.read_cap(cap)
    
    file_save = "live"
    result = cv2.VideoWriter(f'{file_save}.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, (int(cap.get(3)), int(cap.get(4)))) 

    start = time.time()
    while True:
        input_image, display_image, output_scale = posenet.read_cap(cap)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image})

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.15)

        keypoint_coords *= output_scale

        # TODO this isn't particularly fast, use GL for drawing and display someday...
        overlay_image = posenet.draw_skel_and_kp(
            display_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.15, min_part_score=0.3)
        # setting min_pose_score and min_part_score =0 prints the estimated value

        x = keypoint_coords[0][:,0]
        y = keypoint_coords[0][:,1]

        data = []
        for i in range(len(posenet.PART_NAMES)):
            for j in [x, y]:
                data.append(j[i])
        #print(temp)

        temp = pd.DataFrame(data).T

        prediction_RFC = model_RFC.predict(temp)
        prediction_ANN = ohe_categories[np.argmax(model_ANN.predict(temp))]
        prediction_CNN = "Not Ready yet"
        
        temporary = pd.concat([temporary, temp], ignore_index=True)
        X_CNN = []
        if len(temporary) == 4*30:
            X_CNN.append(temporary.values)
            X_CNN = np.asarray(X_CNN).reshape(-1, 4*30, 34, 1)
            prediction_CNN = ohe_categories[np.argmax(model_CNN.predict(X_CNN))]
            temporary.drop(0, inplace=True)
            
        
        font = cv2.FONT_HERSHEY_SIMPLEX 
        fontscale = 1
        color = (255, 150, 0)   # BGR
        thickness = 1
        
        coords_RFC = (0, 20)
        coords_ANN = (0, 50)
        coords_CNN = (0, 85)
        
        cv2.putText(overlay_image, 'RFC = '+prediction_RFC[0], coords_RFC, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'ANN = '+prediction_ANN, coords_ANN, font, fontscale, color, thickness)
        cv2.putText(overlay_image, 'CNN = '+prediction_CNN, coords_CNN, font, fontscale, color, thickness)
        
        cv2.imshow('posenet', overlay_image)
        
        result.write(overlay_image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            result.release()
            break

# print(cap.get(cv2.CAP_PROP_FPS)) -> frame rate
print(f'time taken = {(time.time() - start)/60} mins')


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurre

[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Paralle

[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Paralle

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurre

[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finish

...

[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Paralle

[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Paralle

[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Paralle

time taken = 1.720144502321879 mins


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 488 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:    0.0s finished


In [8]:
model = tf.keras.models.load_model('ANN')
temp = [258.8809064423327, 1096.1560963877098, 247.6743717696233,
       1103.16589102719, 237.49869866443905, 1111.8137990339578,
       211.85583551118512, 1091.1299527575502, 197.20654406924518,
       1082.078272799418, 240.85130642588032, 1042.566650771797,
       224.6896357542929, 1012.5111997844092, 290.9416211163948,
       1054.6050492941076, 324.71345306269507, 967.8415862775054,
       305.6517183036645, 1068.0997568885393, 379.74106977782867,
       972.2731288460247, 355.97467642055307, 1028.445288295582,
       314.87228964965647, 906.4956276720152, 445.0320768951543,
       1030.3417711458944, 406.206993282916, 952.7757588370903,
       535.5605948946843, 1017.1907375791313, 486.6165264304237,
       951.2660654441721]

temp = pd.DataFrame(temp).T
x = model.predict(temp)

print(ohe_categories[np.argmax(x)])

pushups


In [9]:
temporary

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,86.568170,252.842766,70.756901,280.459434,69.476841,233.502669,100.719317,316.981651,101.612092,208.778213,...,462.483522,205.327519,472.992519,303.161434,478.174779,204.951579,476.339437,301.533296,475.651264,206.497350
1,90.158243,249.725802,67.155555,279.157799,69.393860,230.406050,103.322242,316.041491,99.055712,205.791525,...,457.826704,207.799996,466.835413,336.568854,469.437675,223.227676,475.854140,349.262034,474.634337,240.503942
2,88.580588,250.466915,67.225577,279.208028,67.936255,228.383214,104.325963,315.158716,100.122855,207.884923,...,458.259313,208.025894,471.469353,351.935312,469.683145,207.308907,476.412352,349.167455,475.073717,223.777674
3,88.364343,248.765389,67.343392,277.710222,69.570829,229.173715,103.316892,314.587350,98.796143,204.906966,...,458.201290,206.688462,466.603484,337.281978,471.313741,207.054321,476.347878,349.126588,474.628894,208.115584
4,90.516311,249.982037,67.313152,277.822897,69.904666,228.150213,102.608950,313.886445,97.340459,206.537258,...,457.765202,208.188583,469.336622,352.732530,470.410101,223.484264,476.839699,367.080199,474.464134,240.005232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,142.985618,286.992875,130.882534,303.758300,126.194778,272.056188,150.123669,321.431371,143.926864,245.865817,...,460.602927,211.330185,476.676692,366.979377,475.760640,207.010764,476.600661,365.170592,475.141280,207.897697
78,147.004948,282.603818,131.061176,299.571409,129.864808,265.561463,146.202595,319.831391,143.661402,241.107068,...,462.579169,225.308913,477.230049,366.677045,474.836105,223.027432,477.134062,366.061254,474.821520,224.287877
79,150.785067,284.624885,131.766485,301.546163,132.386471,265.242311,143.700620,321.371438,146.026827,243.025350,...,459.622906,196.600711,476.077861,367.157008,474.284394,191.765826,476.782426,366.728283,474.972400,192.460597
80,152.517790,285.675025,133.145623,302.457338,133.442965,266.752769,145.539320,324.195216,144.611493,241.452144,...,460.275286,210.279434,476.065865,366.100955,474.921524,207.350727,477.371651,366.463231,474.657858,208.366241
